In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.volatility import BollingerBands,KeltnerChannel
from ta.trend import MACD

# Download data for NASDAQ index
nasdaq_data = yf.download("^IXIC", start="2010-01-01", end="2023-05-01")
nasdaq_close = nasdaq_data['Close']

# Download data for NSE index
nse_data = yf.download("^NSEI", start="2010-01-01", end="2023-05-01")
nse_close = nse_data['Close']
common_dates_data = pd.merge(nasdaq_data, nse_data, on='Date', how='inner')


# Combine the returns into a single DataFrame
#combined_returns.columns = ['NASDAQ_Returns', 'NSE_Returns']

# Drop rows with missing values
common_dates_data = common_dates_data.dropna()

# Calculate the correlation coefficient
correlation_coef = common_dates_data["Close_x"].corr(common_dates_data['Close_y'])

print("Correlation coefficient between NASDAQ and NSE returns:", correlation_coef)

# Check the direction of the relationship
if correlation_coef > 0:
    print("The relationship between NASDAQ and NSE returns is positive.")
elif correlation_coef < 0:
    print("The relationship between NASDAQ and NSE returns is negative.")
else:
    print("There is no linear relationship between NASDAQ and NSE returns.")




for i in range(1, 21):
    laging_nse = pd.DataFrame()
    laging_nse["lead_nasdaq_returns"] = common_dates_data["Close_x"].shift(-i)
    laging_nse["norm_nse_returns"] = common_dates_data["Close_y"]
    laging_nse = laging_nse.dropna()
    
    correlation_coef_laging_nse = laging_nse["norm_nse_returns"].corr(laging_nse["lead_nasdaq_returns"])
    print(f"Correlation coefficient for laging nse {i}: {correlation_coef_laging_nse}")

for i in range(1, 21):
    leading_nse = pd.DataFrame()
    leading_nse["lag_nasdaq_returns"] = common_dates_data["Close_x"].shift(i)
    leading_nse["norm_nse_returns"] = common_dates_data["Close_y"]
    leading_nse = leading_nse.dropna()
    
    correlation_coef_leading_nse = leading_nse["norm_nse_returns"].corr(leading_nse["lag_nasdaq_returns"])
    print(f"Correlation coefficient for leading nse {i}: {correlation_coef_leading_nse}")




# Calculate Bollinger Bands
bollinger_bands = BollingerBands(close=nse_close, window=14, window_dev=3)
nse_data['BB_Middle'] = bollinger_bands.bollinger_mavg()
nse_data['BB_Upper'] = bollinger_bands.bollinger_hband()
nse_data['BB_Lower'] = bollinger_bands.bollinger_lband()

# Calculate MACDA
macd = MACD(close=nse_close, window_slow=52, window_fast=24, window_sign=9)
nse_data['MACD'] = macd.macd()
nse_data['Signal_Line'] = macd.macd_signal()

print(nse_data[['Close', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'MACD', 'Signal_Line']])


keltner_channel = KeltnerChannel(high=nse_data['High'], low=nse_data['Low'], close=nse_data['Close'], window=25, window_atr=10)
nse_data['KC_Middle'] = keltner_channel.keltner_channel_mband()
nse_data['KC_Upper'] = keltner_channel.keltner_channel_hband()
nse_data['KC_Lower'] = keltner_channel.keltner_channel_lband()


nse_data = nse_data.dropna()
nse_data.reset_index()
#calculating accuracy of keltner channel
signals_Sell = pd.DataFrame()
signals_Sell = (nse_data[nse_data["High"]>=nse_data["KC_Upper"]])
        

signals_Sell.loc[:, 'Signal'] = 'sell'


signals_buy = pd.DataFrame()
signals_buy = (nse_data[nse_data["Low"]<=nse_data["KC_Lower"]])
        

signals_buy.loc[:, 'Signal'] = 'buy'



signals = pd.concat([signals_Sell, signals_buy])
signals = signals.sort_values('Date')




signals.reset_index()

signals = pd.concat([signals_Sell, signals_buy])
combined_signals = signals.sort_index()
combined_signals.reset_index(drop=True, inplace=True)
count1 = 0
count0 = 0

for i in range(0, len(combined_signals)-1):
    if combined_signals.loc[i, 'Signal'] == 'sell':
        temp = i
        while temp != len(combined_signals)-1 and combined_signals.loc[temp, 'Signal'] == 'sell':
            temp += 1
        if combined_signals.loc[temp, 'High'] <= combined_signals.loc[i, 'Low']:
            count1 = count1 + 1
        else:
            count0 = count0 + 1
    if combined_signals.loc[i, 'Signal'] == 'buy':
        temp = i
        while temp != len(combined_signals)-1 and combined_signals.loc[temp, 'Signal'] == 'buy':
            temp += 1
        if combined_signals.loc[i, 'High'] <= combined_signals.loc[temp, 'Low']:
            count1 = count1 + 1
        else:
            count0 = count0 + 1

accuracykc = (count1/(count0+count1))*100
print(f" KC = {accuracykc}")
#calculating accuracy og bollinger_bands
signals_Sell = pd.DataFrame()
signals_Sell = (nse_data[nse_data["High"]>=nse_data["BB_Upper"]])
        

signals_Sell.loc[:, 'Signal'] = 'sell'


signals_buy = pd.DataFrame()
signals_buy = (nse_data[nse_data["Low"]<=nse_data["BB_Lower"]])
        

signals_buy.loc[:, 'Signal'] = 'buy'



signals = pd.concat([signals_Sell, signals_buy])
signals = signals.sort_values('Date')




signals.reset_index()

signals = pd.concat([signals_Sell, signals_buy])
combined_signals = signals.sort_index()
combined_signals.reset_index(drop=True, inplace=True)
count1 = 0
count0 = 0

for i in range(0, len(combined_signals)-1):
    if combined_signals.loc[i, 'Signal'] == 'sell':
        temp = i
        while temp != len(combined_signals)-1 and combined_signals.loc[temp, 'Signal'] == 'sell':
            temp += 1
        if combined_signals.loc[temp, 'High'] <= combined_signals.loc[i, 'Low']:
            count1 = count1 + 1
        else:
            count0 = count0 + 1
    if combined_signals.loc[i, 'Signal'] == 'buy':
        temp = i
        while temp != len(combined_signals)-1 and combined_signals.loc[temp, 'Signal'] == 'buy':
            temp += 1
        if combined_signals.loc[i, 'High'] <= combined_signals.loc[temp, 'Low']:
            count1 = count1 + 1
        else:
            count0 = count0 + 1

accuracybb = (count1/(count0+count1))*100
print(f"bb = {accuracybb}")

#calculating accuracy of macd
signals_Sell = pd.DataFrame()
signals_Sell = (nse_data[nse_data["Signal_Line"]>=nse_data["MACD"]])
        

signals_Sell.loc[:, 'Signal'] = 'sell'


signals_buy = pd.DataFrame()
signals_buy = (nse_data[nse_data["Signal_Line"]<=nse_data["MACD"]])
        

signals_buy.loc[:, 'Signal'] = 'buy'



signals = pd.concat([signals_Sell, signals_buy])
signals = signals.sort_values('Date')




signals.reset_index()

signals = pd.concat([signals_Sell, signals_buy])
combined_signals = signals.sort_index()
combined_signals.reset_index(drop=True, inplace=True)
count1 = 0
count0 = 0

for i in range(0, len(combined_signals)-1):
    if combined_signals.loc[i, 'Signal'] == 'sell':
        temp = i
        while temp != len(combined_signals)-1 and combined_signals.loc[temp, 'Signal'] == 'sell':
            temp += 1
        if combined_signals.loc[temp, 'Low'] >= combined_signals.loc[i, 'High']:
            count1 = count1 + 1
        else:
            count0 = count0 + 1
    if combined_signals.loc[i, 'Signal'] == 'buy':
        temp = i
        while temp != len(combined_signals)-1 and combined_signals.loc[temp, 'Signal'] == 'buy':
            temp += 1
        if combined_signals.loc[i, 'Low'] >= combined_signals.loc[temp, 'High']:
            count1 = count1 + 1
        else:
            count0 = count0 + 1

accuracyMACD = (count1/(count0+count1))*100
print(accuracyMACD)

bollinger_bands = BollingerBands(close=nasdaq_close, window=14, window_dev=3)
nasdaq_data['BB_Middle'] = bollinger_bands.bollinger_mavg()
nasdaq_data['BB_Upper'] = bollinger_bands.bollinger_hband()
nasdaq_data['BB_Lower'] = bollinger_bands.bollinger_lband()

macd = MACD(close=nasdaq_close, window_slow=52, window_fast=24, window_sign=9)
nasdaq_data['MACD'] = macd.macd()
nasdaq_data['Signal_Line'] = macd.macd_signal()

print(nasdaq_data[['Close', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'MACD', 'Signal_Line']])

keltner_channel = KeltnerChannel(high=nasdaq_data['High'], low=nasdaq_data['Low'], close=nasdaq_data['Close'], window=25, window_atr=10)
nasdaq_data['KC_Middle'] = keltner_channel.keltner_channel_mband()
nasdaq_data['KC_Upper'] = keltner_channel.keltner_channel_hband()
nasdaq_data['KC_Lower'] = keltner_channel.keltner_channel_lband()

nasdaq_data = nasdaq_data.dropna()
nasdaq_data.reset_index()

#calculating accuracy of keltner channel
signals_Sell = pd.DataFrame()
signals_Sell = (nasdaq_data[nasdaq_data["High"]>=nasdaq_data["KC_Upper"]])
        

signals_Sell.loc[:, 'Signal'] = 'sell'


signals_buy = pd.DataFrame()
signals_buy = (nasdaq_data[nasdaq_data["Low"]<=nasdaq_data["KC_Lower"]])
        

signals_buy.loc[:, 'Signal'] = 'buy'



signals_KC = pd.concat([signals_Sell, signals_buy])
signals_KC = signals.sort_values('Date')
#calculating accuracy og bollinger_bands
signals_Sell = pd.DataFrame()
signals_Sell = (nasdaq_data[nasdaq_data["High"]>=nasdaq_data["BB_Upper"]])
        

signals_Sell.loc[:, 'Signal'] = 'sell'


signals_buy = pd.DataFrame()
signals_buy = (nasdaq_data[nasdaq_data["Low"]<=nasdaq_data["BB_Lower"]])
        

signals_buy.loc[:, 'Signal'] = 'buy'



signals_BB = pd.concat([signals_Sell, signals_buy])
signals_BB = signals.sort_values('Date')
#calculating accuracy of macd
signals_Sell = pd.DataFrame()
signals_Sell = (nasdaq_data[nasdaq_data["Signal_Line"]>=nasdaq_data["MACD"]])
        

signals_Sell.loc[:, 'Signal'] = 'sell'


signals_buy = pd.DataFrame()
signals_buy = (nasdaq_data[nasdaq_data["Signal_Line"]<=nasdaq_data["MACD"]])
        

signals_buy.loc[:, 'Signal'] = 'buy'



signals_MACD = pd.concat([signals_Sell, signals_buy])
signals_MACD = signals.sort_values('Date')




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Correlation coefficient between NASDAQ and NSE returns: 0.9513138758576783
The relationship between NASDAQ and NSE returns is positive.
Correlation coefficient for laging nse 1: 0.9507737700328763
Correlation coefficient for laging nse 2: 0.950286280116005
Correlation coefficient for laging nse 3: 0.9497767429972327
Correlation coefficient for laging nse 4: 0.9492839758160373
Correlation coefficient for laging nse 5: 0.948751310214809
Correlation coefficient for laging nse 6: 0.9482362261854125
Correlation coefficient for laging nse 7: 0.9477096919386553
Correlation coefficient for laging nse 8: 0.9471446302076983
Correlation coefficient for laging nse 9: 0.9466095202451897
Correlation coefficient for laging nse 10: 0.9460633295628981
Correlation coefficient for laging nse 11: 0.9455306106805285
Correlation coefficient for laging nse 12: 0.9449920822

C:\Users\Piyush Patil\AppData\Local\Temp\ipykernel_15864\2216216721.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_Sell.loc[:, 'Signal'] = 'sell'
C:\Users\Piyush Patil\AppData\Local\Temp\ipykernel_15864\2216216721.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_buy.loc[:, 'Signal'] = 'buy'


 KC = 43.17082247860067
bb = 60.97560975609756


C:\Users\Piyush Patil\AppData\Local\Temp\ipykernel_15864\2216216721.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_Sell.loc[:, 'Signal'] = 'sell'
C:\Users\Piyush Patil\AppData\Local\Temp\ipykernel_15864\2216216721.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_buy.loc[:, 'Signal'] = 'buy'
C:\Users\Piyush Patil\AppData\Local\Temp\ipykernel_15864\2216216721.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

41.77057356608479
                   Close     BB_Middle      BB_Upper      BB_Lower   
Date                                                                 
2010-01-04   2308.419922           NaN           NaN           NaN  \
2010-01-05   2308.709961           NaN           NaN           NaN   
2010-01-06   2301.090088           NaN           NaN           NaN   
2010-01-07   2300.050049           NaN           NaN           NaN   
2010-01-08   2317.169922           NaN           NaN           NaN   
...                  ...           ...           ...           ...   
2023-04-24  12037.200195  12084.574986  12285.652594  11883.497378   
2023-04-25  11799.160156  12061.205706  12355.763797  11766.647614   
2023-04-26  11854.349609  12051.026367  12383.709041  11718.343693   
2023-04-27  12142.240234  12054.903530  12394.040827  11715.766232   
2023-04-28  12226.580078  12065.062081  12429.032349  11701.091814   

                  MACD  Signal_Line  
Date                             

C:\Users\Piyush Patil\AppData\Local\Temp\ipykernel_15864\2216216721.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_Sell.loc[:, 'Signal'] = 'sell'
C:\Users\Piyush Patil\AppData\Local\Temp\ipykernel_15864\2216216721.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals_buy.loc[:, 'Signal'] = 'buy'
C:\Users\Piyush Patil\AppData\Local\Temp\ipykernel_15864\2216216721.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index